### Meta

In [5]:
# Autoreload modules without having to restart the notebook kernel.
%load_ext autoreload
%autoreload 2


# Plotting code stolen from Georg's notebook.
import matplotlib.pyplot as plt

%matplotlib inline
font = {'family': 'DejaVu Sans',
        'weight': 'bold',
        'size': 32}
plt.rc('font', **font)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Imports

In [6]:
import sys

sys.path.insert(1, "..\\")

#import markov_clustering as mcl

# Personal libraries
import lib.cluster
import lib.files
import lib.graph

In [3]:
### Constants

In [12]:
# ICP55's inviable neighbour.
YHC1 = '4932.YLR298C'

In [3]:
network = lib.graph.read_weighted_edgelist(lib.files.make_filepath_to_networks('icp55-cc-900-inv.txt'))

# GARBAGE ZONE

Nothing past here is clean or makes readable sense.

In [ ]:
clusters = lib.cluster.read_csv(lib.files.make_filepath_to_mcl_clusters("icp55-cc-with-inviable-proteins.csv"))
clusters = sorted(clusters, key=len, reverse=True)

In [8]:
clusters = clusters[:3]

In [9]:
lib.cluster.write_to_file(lib.files.make_filepath_to_mcl_clusters("largest-3-for-nova.csv"), clusters)

In [11]:
import networkx as nx
network = lib.graph.read_weighted_edgelist(lib.files.make_filepath_to_networks('icp55-cc-900-inv.txt'))
clusters = [network.subgraph(cluster) for cluster in clusters]

In [17]:
betweeness_centralities = [nx.algorithms.centrality.betweenness_centrality(cluster) for cluster in clusters]
top_5 = []

for i in range(3):
        cluster_centrality = betweeness_centralities[i].items()
        cluster_centrality = sorted(cluster_centrality, key=lambda x: x[1], reverse=True)
        top_5.append(cluster_centrality[:5])

In [20]:
minimised_clusters = [[x[0] for x in y] for y in top_5]
lib.cluster.write_to_file(lib.files.make_filepath_to_clusters('top_5_of_largest_3_clusters.csv'), minimised_clusters)